In [25]:
import os
import re
import pickle
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

def load_documents(input_path: str) -> List[Document]:
    """
    저장된 pickle 파일에서 List[Document]를 로드합니다.
    
    Args:
        input_path (str): 문서가 저장된 디렉토리 경로 또는 파일 경로
        
    Returns:
        List[Document]: 로드된 문서 리스트
    """
    # 디렉토리 경로인 경우 documents.pkl 파일을 찾음
    if os.path.isdir(input_path):
        docs_file_path = os.path.join(input_path, "documents.pkl")
    else:
        docs_file_path = input_path
    
    if not os.path.exists(docs_file_path):
        raise FileNotFoundError(f"문서 파일을 찾을 수 없습니다: {docs_file_path}")
    
    with open(docs_file_path, "rb") as f:
        docs = pickle.load(f)
    
    print(f"✅ 문서 로드 완료: {docs_file_path}")
    print(f"📄 로드된 문서 수: {len(docs)}")
    
    return docs

In [28]:
###### Retriever Check from FAISS Vector DB ######

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever

docs_markdown = load_documents("models/faiss_vs_rag_iap_v11_cr2_bge")
fixed_model_name = "bge-m3:latest"
output_dir = "models/faiss_vs_rag_iap_v11_cr2_bge"

# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model=fixed_model_name)

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path=output_dir,
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 20, "fetch_k": 70, "lambda_mult": 0.7}
    # search_type="similarity",
    # search_kwargs={"k": 50}
)

bm25 = BM25Retriever.from_documents(
    docs_markdown,
    bm25_params={"k1": 1.5, "b": 0.75}
)

bm25.k = 20

ensembled_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],
    weights=[0.5, 0.5]
)


res = ensembled_retriever.invoke(
    # "원스토어 인앱결제의 PNS의 개념을 설명해주세요"
    "PNS 메시지 규격의 purchaseState는 어떤 값으로 구성되나요?"
)

# res = bm25.invoke(
#     # "PNS의 메세지 규격을 설명해주세요",
#     "PNS 메시지 규격의 purcahseState는 어떤 값으로 구성되나요?"
# )

# res = bm25.invoke(
#     # "원스토어 인앱결제의 PNS의 개념을 설명해주세요",
#     "PNS 메시지 규격의 purcahseState는 어떤 값으로 구성되나요?"
# )

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    # print(doc.metadata)
    # print('-' * 40)
    idx += 1

✅ 문서 로드 완료: models/faiss_vs_rag_iap_v11_cr2_bge/documents.pkl
📄 로드된 문서 수: 495
검색된 문서 수: 34
--- doc_index: 0 ---
[제목]: 09. 원스토어 인앱결제 릴리즈 노트 / **원스토어 인앱결제 라이브러리 API V6(SDK V19) 출시** <a href="#id-09.-apiv6-sdkv19" id="id-09.-apiv6-sdkv19"></a> / PNS 메시지 규격 변경  <a href="#id-09.-pns" id="id-09.-pns"></a>

[요약]: **요약:**

결제 API 문서는 원화 외 통화 지원을 위해 결제금액과 결제수단 금액의 데이터 타입을 String으로 변경하고, 통화코드(priceCurrencyCode)를 추가했습니다. 응답 규격에는 purchaseToken, environment, marketCode 필드가 추가되었습니다. 주요 기능은 주문 정보 조회, 결제 처리, 결제 결과 확인으로 구성되어 있으며, 다양한 통화 및 환경 지원을 통해 글로벌 결제 시스템을 강화하고 있습니다. **결제API, 글로벌결제, 통화지원**

[원문]: * 원화 외의 통화를 지원하기 위하여 결제금액(price)의 데이터 타입이 Number에서 String으로 변경하였습니다.
* 원화 외의 통화를 지원하기 위하여 결제금액의 통화코드(priceCurrencyCode)를 추가하였습니다.
* 원화 외의 통화를 지원하기 위하여 결제수단별금액(amount)의 데이터 타입이 Number에서 String으로 변경하였습니다.
* 응답 규격에 purchaseToken, environment, marketCode 필드가 추가되었습니다.&#x20;  
상세한 규격은 PNS 메시지 상세 변경 내역에서 확인 할 수 있습니다. &#x20;
--- doc_index: 1 ---
[제목]: 07. PNS(Payment Notification Service) 이용하기 / Notification 전송 정책